
# Translation English Indonesia (RNN) -> Evaluation

---

### Masalah yang ingin diselesaikan 
Masalah translasi teks dari Bahasa Inggris ke Indonesia 

### Data yang Dipilih

Data yang kami dapatkan dari https://github.com/lppier/Seq2Seq_Eng2Indo-Translation, tetapi karena awalnya masih kurang jadi kami menambahkan dataset tambahan https://github.com/prasastoadi/parallel-corpora-en-id/ (IWSLT17.TED.tst2017plus.en-id.tok, PANL-BPPT-INT-EN-150Kw.tok, PANL-BPPT-SPO-EN-100Kw.tok)

### Arsitektur dan Model yang Dipilih
- Arsitektur: RNN
- Model: Seq 2 Seq 

## Import All Libraries

In [1]:
import pickle
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from nltk import word_tokenize

## Setup for tokenization

In [2]:
MIN_LENGTH = 4 # define min words
MAX_LENGTH = 102 # define max words
START, START_IDX = '<s>',  0
END, END_IDX = '</s>', 1
UNK, UNK_IDX = 'UNK', 2

SOS_token = START_IDX
EOS_token = END_IDX

print(torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()

# Lets load our dictionaries.
f_eng = open('vocabs/simple_english_vocab.Dictionary.pkl', 'rb')
english_vocab = pickle.load(f_eng)

f_ind = open('vocabs/simple_indo_vocab.Dictionary.pkl', 'rb')
indo_vocab = pickle.load(f_ind)

2.0.1


### Penjelasan:
Pada bagian ini kita akan melakukan setting untuk tokenization dari input yang ada dan kita akan load dictionary tokenizer yang kita dapat dari training sebelumnya


## Define Encoder and Decoder 

In [3]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):  # Add max_length as a parameter
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length  # Update max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)  # Update attention layer
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### Penjelasan:
EncoderRNN memproses input menggunakan embedding dan GRU (Gated Recurrent Unit), sedangkan AttnDecoderRNN berfungsi sebagai decoder dengan mekanisme attention untuk menghasilkan output berdasarkan input, hidden state, dan output dari encoder.

## Convert the Sentences to Vector

In [4]:
#mengonversi kalimat menjadi vektor 
def vectorize_sent(sent, vocab):
    return vocab.doc2idx([START] + word_tokenize(sent.lower()) + [END], unknown_word_index=2)

#mengonversi vektor ke dalam bentuk tensor pytorch untuk input model 
def variable_from_sent(sent, vocab):
    vsent = vectorize_sent(sent, vocab)
    # print(vsent)
    result = Variable(torch.LongTensor(vsent).view(-1, 1))
    # print(result)
    return result.cuda() if use_cuda else result

## Make Evaluation Function

In [5]:
#lakukan evaluasi pada model
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        # input_tensor = tensorFromSentence(input_lang, sentence)
        input_tensor = variable_from_sent(sentence, english_vocab)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('</s>')
                break
            else:
                decoded_words.append(indo_vocab.id2token[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [6]:
hidden_size = 512
encoder = EncoderRNN(len(english_vocab), hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, len(indo_vocab), dropout_p=0.5).to(device)

## Load a network weight snapshot

In [7]:
ENCODER_PATH = 'results/encoder-{}.pth'
DECODER_PATH = 'results/decoder-{}.pth'

EPOCH_NO = 75000

encoder.load_state_dict(torch.load(ENCODER_PATH.format(EPOCH_NO), map_location=device))
encoder.eval()

EncoderRNN(
  (embedding): Embedding(18990, 512)
  (gru): GRU(512, 512)
)

In [8]:
attn_decoder.load_state_dict(torch.load(DECODER_PATH.format(EPOCH_NO),map_location=device))
attn_decoder.eval()

AttnDecoderRNN(
  (embedding): Embedding(18722, 512)
  (attn): Linear(in_features=1024, out_features=102, bias=True)
  (attn_combine): Linear(in_features=1024, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (gru): GRU(512, 512)
  (out): Linear(in_features=512, out_features=18722, bias=True)
)

## Load a Mini Evaluation

In [9]:
input_sentence = "Tom is a good man"
output_words, attentions = evaluate(encoder, attn_decoder, input_sentence)
output_sentence = ' '.join(output_words)
print('>', input_sentence)
print('<', output_sentence)

> Tom is a good man
< <s> tom adalah seorang yang baik . </s>


## Load an array of network weights snapshots

In [10]:
ENCODER_PATH = 'results/encoder-{}.pth'
DECODER_PATH = 'results/decoder-{}.pth'

EPOCH_NO = [75000]

def load_weights(encoder, decoder, epoch_no, device):
    encoder.load_state_dict(torch.load(ENCODER_PATH.format(epoch_no),map_location=device))
    decoder.load_state_dict(torch.load(DECODER_PATH.format(epoch_no),map_location=device))
    return encoder, decoder


# Input Validation to make sure if the inputs between the min length and max length
def input_validation(input_text, english_vocab):
    max_words_required = MAX_LENGTH - 2
    min_words_required = MIN_LENGTH
    input_tokenized = word_tokenize(input_text.lower())
    final_text = None
    message = ""
    if not min_words_required <= len(input_tokenized) <= max_words_required:
        message = "The input sentence should be between {} and {} words".format(MIN_LENGTH, MAX_LENGTH - 2)
    else:
        input_ids = english_vocab.doc2idx(input_tokenized)
        unknown_tokens = []
        for key, val in enumerate(input_ids):
            if val == -1:
                unknown_token = input_tokenized[key]
                unknown_tokens.append(unknown_token)
                input_tokenized[key] = UNK.lower()
                print("'{}' is not found in the english corpus".format(unknown_token))
        final_text = " ".join(input_tokenized)
    return final_text, message


In [29]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: ipywidgets in c:\users\elisa\anaconda3\envs\deeplearning\lib\site-packages (8.0.4)



## Evaluation

In [12]:
import ipywidgets as widgets

input_text = widgets.Text(
    value='',
    placeholder='Type something',
    description='English:',
    disabled=False
)

button = widgets.Button(
    description='Translate',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Translate',
    icon='check'
)

def on_button_clicked(b):
    eng2indo_translation(input_text.value, encoder, attn_decoder, english_vocab)

button.on_click(on_button_clicked)

display(input_text)
display(button)

def eng2indo_translation(text, encoder, decoder, english_vocab):
    sentences = text.split('.')  # Split input text into sentences based on "."
    translated_sentences = []

    for sentence in sentences:
        final_text, message = input_validation(sentence.strip(), english_vocab)

        if final_text:
            print('>', sentence)
            print('>>', final_text)

            for epoch in EPOCH_NO:
                encoder, decoder = load_weights(encoder, decoder, epoch, device)
                output_words, attentions = evaluate(encoder, decoder, final_text)
                output_sentence = ' '.join(output_words).replace('<s>', '').replace('</s>', '')
                translated_sentences.append(output_sentence)

    # Concatenate translated sentences
    final_output = ' '.join(translated_sentences)
    print("Final Translation:", final_output)

    if message:
        print("Validation Message:", message)


Text(value='', description='English:', placeholder='Type something')

Button(button_style='info', description='Translate', icon='check', style=ButtonStyle(), tooltip='Translate')

In [12]:
eng2indo_translation("she is standing there .", encoder, attn_decoder, english_vocab)
eng2indo_translation("he is a bad man .", encoder, attn_decoder, english_vocab)
eng2indo_translation("he wants to sleep .", encoder, attn_decoder, english_vocab)
eng2indo_translation("i can't see you crying .", encoder, attn_decoder, english_vocab)
eng2indo_translation("my dog is running around .", encoder, attn_decoder, english_vocab)
eng2indo_translation("it is very popular .", encoder, attn_decoder, english_vocab)
eng2indo_translation("she speaks american english to tom's father .", encoder, attn_decoder, english_vocab)
eng2indo_translation("please eat lunch in the afternoon .", encoder, attn_decoder, english_vocab)
eng2indo_translation("i see red roses in the garden .", encoder, attn_decoder, english_vocab)

> she is standing there 
>> she is standing there
Final Translation:  dia ada di sana . 
Validation Message: The input sentence should be between 4 and 100 words
> he is a bad man 
>> he is a bad man
Final Translation:  dia adalah seorang yang yang sangat 
Validation Message: The input sentence should be between 4 and 100 words
> he wants to sleep 
>> he wants to sleep
Final Translation:  dia ingin tidak lagi . 
Validation Message: The input sentence should be between 4 and 100 words
'n't' is not found in the english corpus
> i can't see you crying 
>> i ca unk see you crying
Final Translation:  aku sudah makan kamu kamu . 
Validation Message: The input sentence should be between 4 and 100 words
> my dog is running around 
>> my dog is running around
Final Translation:  saya saya berada di luar . 
Validation Message: The input sentence should be between 4 and 100 words
> it is very popular 
>> it is very popular
Final Translation:  ini benar benar . 
Validation Message: The input sente

## Komitmen Integritas
Di hadapan TUHAN yang hidup, saya menegaskan bahwa saya tidak memberikan maupun
menerima bantuan apapun—baik lisan, tulisan, maupun elektronik—di dalam ujian ini selain
daripada apa yang telah diizinkan oleh pengajar, dan tidak akan menyebarkan baik soal
maupun jawaban ujian kepada pihak lain.